# develop and verify bc procedures

p3_yifan?

In [2]:
import time
# from datetime import datetime
# import glob, os
import xarray as xr
import dask
import numpy as np
from random import randrange
import gc
import sys
import glob
# sys.path.append('/glade/u/home/bkruyt/libraries')
# sys.path.append('/glade/u/home/bkruyt/libraries/st_lines')
# sys.path
import quantile_mapping # from same folder.

### Functions

In [ ]:
test=True
def get_livneh(icar_1file):
    """get livneh data, cropped to the icar grid (defined by icar_1file)"""

    t0 = time.time()
    print("\n- - - - - - - - - -   opening livneh files - - - - - - - - - - - - ")

    ### Livneh Precipitation [mm]
    files = glob.glob("/glade/campaign/ral/hap/common/Livneh_met_updated/precip/livneh_unsplit_precip.2021-05-02.19[5-9]*.nc")
    files.extend(glob.glob("/glade/campaign/ral/hap/common/Livneh_met_updated/precip/livneh_unsplit_precip.2021-05-02.20*.nc"))
    files.sort()
    if test:
        livneh = xr.open_mfdataset(files[:10])  # speed up for testing
        print('   ! TEST : ',len(files[:10]), " Livneh precipiation files(years)" ) # the number of files (years)
    else:
        print('   ',len(files), " Livneh precipiation files(years)" ) # the number of files (years)
        livneh = xr.open_mfdataset(files, parallel=True)

    # # # crop livneh to ICAR bounds:
    # buff=10.5
    max_lat=icar_1file.lat.max().values#+buff
    min_lat=icar_1file.lat.min().values#-buff
    max_lon=icar_1file.lon.max().values#+buff
    min_lon=icar_1file.lon.min().values#-buff

    LatIndexer, LonIndexer = 'lat', 'lon'

    livneh = livneh.sel(**{LatIndexer: slice(min_lat, max_lat ),
                            LonIndexer: slice(min_lon, max_lon)})

    # print(livneh)

    livneh_pr   = livneh["PRCP"]#.load()

    # clean up some data; correct time dimension:
    if 'Time' in livneh_pr.dims:
        livneh_pr = livneh_pr.rename( {'Time':'time'})
    if not ('time' in livneh_pr.dims):
        print(' warning ' )

    print("   ", time.time() - t0)
    return livneh_pr #, livneh_tmin, livneh_tmax

In [5]:
noise_path  = "/glade/derecho/scratch/bkruyt/CMIP6"
NOISE_u     = xr.open_dataset(f"{noise_path}/uniform_noise_480_480.nc" )
noise_val   = 0.01


def add_noise(df, noise_val=noise_val, random=True):
    """Add uniform noise to a 2D dataset df."""

    with dask.config.set(**{'array.slicing.split_large_chunks': True}):

        u_noise = NOISE_u.uniform_noise  #.load() # 55000 x 480 x480

        # Add noise to (daily) input data:
        # to avoid taking the same noise values every time, we add a random int.
        t = len(df.time)
        if random:
            r = randrange(1000)
        else:
            r = 0
        noise_arr = noise_val * u_noise[r:(r+t) , :df.shape[1], :df.shape[2] ]
        df = xr.where( df>0, df + noise_val, noise_arr.values)

        return df


### Test

In [6]:
###########
CMIP="CMIP6"
model="MIROC-ES2L"
scen_load ="hist"
base_in  = f"/glade/derecho/scratch/bkruyt/{CMIP}/WUS_icar_LivGrd2"


#### Get input data

In [7]:
start_year=1950 ; end_year=1951 ; dt="3hr"

files=[]
for y in range(int(start_year), int(end_year)+1) :

    if CMIP=="CMIP6":
        files.extend( glob.glob(f'{base_in}/{model}_hist/{dt}/icar_*_{y}*.nc') )
        files.extend( glob.glob(f'{base_in}/{model}_{scen_load}_2004/{dt}/icar_*_{y}*.nc') )
        files.extend( glob.glob(f'{base_in}/{model}_{scen_load}_2049/{dt}/icar_*_{y}*.nc') )

pr   =  xr.open_mfdataset( files )['precip_dt']

#### Get ref and obs data

In [8]:


icar_1_for_grid = pr #xr.open_mfdataset( files[0] )
livneh_pr = get_livneh(icar_1file=icar_1_for_grid.isel(time=0))
dsObs     = livneh_pr.load()



- - - - - - - - - -   opening livneh files - - - - - - - - - - - - 
   ! TEST :  10  Livneh precipiation files(years)
    0.716710090637207


In [9]:
tdelta_int=3
if tdelta_int<24:
    print("   making daily precip")
    daily = pr.resample(time='1D').sum(dim='time').load()
else:
    daily = pr.load()

   making daily precip


In [20]:
dsRef   = daily.copy()  # for test only!
daily_n = add_noise(daily, noise_val=noise_val, random=True)
dsRef_n = add_noise(dsRef, noise_val=noise_val, random=True)

In [27]:
# reduce array shape:
a=150; b=200
daily_n = daily_n.isel(lat=slice(a,b)).isel(lon=slice(a,b))
dsRef_n = dsRef_n.isel(lat=slice(a,b)).isel(lon=slice(a,b))
dsObs   = dsObs.isel(lat=slice(a,b)).isel(lon=slice(a,b))

daily_n.shape, dsRef_n.shape, dsObs.shape

((730, 50, 50), (5840, 50, 50), (3652, 50, 50))

### Bias Correct

In [ ]:
bc_daily = quantile_mapping.quantile_mapping_by_group(
    # def quantile_mapping_by_group(input_data, ref_data, data_to_match, (ref should exclude 5y input)
    daily_n,  dsRef_n, dsObs, # correct to liv grid
    grouper='time.month', detrend=False,  use_ref_data=True,# use_ref_data is not used! set ref_data=None to exclude ref data!
    extrapolate="1to1", n_endpoints=50
    )

In [ ]:
bc_daily

### Reapply to 3hr:

In [ ]:
print("      pr.shape:",pr.shape)
print("      pr == 0 :",np.sum(pr.values==0))
print("   applying correction to ",tdelta_int,"-hourly data")

In [ ]:
# Calculate the correction array
correction = bc_daily.values / daily_n.values
# repeat for nr of timesteps per day:
correction =  np.repeat(correction, (24 / tdelta_int), axis=0)

# Create a mask for elements where daily sum is zero and bc_daily is positive
mask = (daily == 0) & (bc_daily > 0)
mask = np.repeat(mask, (24 / tdelta_int), axis=0)

print(f"      mask shape: {mask.shape}")
print(f"      correction shape: {correction.shape} ") # \n

In [ ]:
bc_tdelta =  bc_daily.interp(time=pr.time)
print(f"      bc_tdelta shape: {bc_tdelta.shape}")

In [ ]:
    # # Apply correction to (all) values
    pr *= correction

    for t in range( pr.shape[0]):
        # where mask, retrun bc_tdelta (syntax: Returns elements from ‘DataArray’, where ‘cond’ is True, otherwise fill in ‘other’. )
        # print(pr[t].shape, mask[t].shape, bc_tdelta[t].shape )
        try:
            pr[t] = pr[t].where(~mask[t], bc_tdelta[t])
        except:
            print(f"   pr.shape: {pr.shape} ")
            print(f"   mask.shape: {mask.shape} ")
            print(f"   bc_tdelta.shape: {bc_tdelta.shape} ")